In [1]:
from music21 import chord
import torch.nn.functional as F
import torch
from synthesizer import Player, Synthesizer, Waveform
from density import density
player_init = False

In [6]:
# set up the target chord and a random input chord
ch = chord.Chord(['F', 'A', 'C5'])
target = torch.tensor([pitch.frequency for pitch in ch.pitches])

# input_chord = [pitch.frequency for pitch in chord.Chord(
#               ['C', 'A', 'A', 'C', 'C']).pitches]

input = torch.tensor(torch.rand((100)) * 400, requires_grad=True)

# set up optimizer and loss
optimizer = torch.optim.Adam([input], lr=1e-1)
loss_fn = torch.nn.L1Loss()

In [7]:
# optimization loop
for i in range(5000):

    optimizer.zero_grad()
    loss = loss_fn(density(input), density(target))
    loss.backward()
    optimizer.step()

In [8]:
input = input.detach().cpu()
target = target.detach().cpu()
# print('input', input)
# print('target', target)
print(f'input density {density(input)}, target density {density(target)}')

input density 1.3033148050308228, target density 1.303205132484436


In [9]:
# play result
if not player_init:
    player = Player()
    player.enumerate_device()
    player.open_stream(device_name="HD-Audio Generic: ALC1220 Analog (hw:1,0)")
    synthesizer = Synthesizer(osc1_waveform=Waveform.sine,
                            osc1_volume=1.0, use_osc2=False)
    player_init = True
player.play_wave(synthesizer.generate_chord(input.tolist(), 2.0))

tensor([251.5066, 222.4198, 282.6039,  59.5248,  69.8243])